<a href="https://colab.research.google.com/github/guanqun-yang/remote-notebooks/blob/master/2024_01_05_AEDDataGenerationProcess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/guanqun-yang/remote-data.git
! git clone https://github.com/autonlab/aqua.git

fatal: destination path 'remote-data' already exists and is not an empty directory.
fatal: destination path 'aqua' already exists and is not an empty directory.
python3: can't open file '/content/setup.py': [Errno 2] No such file or directory


In [ ]:
! pip install -q sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.6 MB/s eta 0:00:00


In [ ]:
import pathlib
import numpy as np
import pandas as pd

from sklearn.metrics import (
    classification_report,
)
from transformers.trainer_utils import (
    set_seed
)
import seaborn as sns
import matplotlib.pyplot as plt
from sentence_transformers import (
    SentenceTransformer,
)

set_seed(42)
base_path = pathlib.Path("/content/remote-data/2024-01-05")

DEBUG:jax._src.path:etils.epath found. Using etils.epath for file I/O.
DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


In [ ]:
from aqua.aqua.evaluation.noise import (
    DissentingLabelNoise,
    DissentingWorkerNoise,
    UniformNoise,
    ClassDependentNoise,
    InstanceDependentNoise,
    AsymmetricNoise,
)

def add_noise(X, y, n_classes, noise_type, noise_rate, annotator_labels=None):
    assert noise_type in ["uniform", "dissenting_worker", "dissenting_label",
                          "asymmetric", "classdependent", "instancedependent"]

    noise_kwargs = {
        "model": None,
        "data": None,
        "batch_size": None,
        "device": None,
    }

    if noise_type == 'uniform':
        noise_model = UniformNoise(n_classes, noise_rate)
    elif noise_type == 'dissenting_worker':
        noise_model = DissentingWorkerNoise(n_classes, noise_rate)
    elif noise_type == 'dissenting_label':
        noise_model = DissentingLabelNoise(n_classes, noise_rate)
    elif noise_type == 'asymmetric':
        noise_model = AsymmetricNoise(n_classes, noise_rate)
    elif noise_type == 'classdependent':
        noise_model = ClassDependentNoise(
            n_classes,
            model=noise_kwargs['model'],
            data=noise_kwargs['data'],
            device=noise_kwargs["device"],
            batch_size=noise_kwargs['batch_size']
        )
    elif noise_type == 'instancedependent':
        noise_model = InstanceDependentNoise(
            n_classes,
            noise_rate=noise_rate,
            model=noise_kwargs['model'],
            data=noise_kwargs['data'],
            device=noise_kwargs["device"],
            batch_size=noise_kwargs['batch_size']
        )
    else:
        return

    if noise_model.multi_annotator:
        noisy_X, noisy_y = noise_model.add_noise(X=X, y=y, annotator_y=annotator_labels)
    else:
        noisy_X, noisy_y = noise_model.add_noise(X=X, y=y)

    return noisy_X, noisy_y

In [ ]:
label_columns = [
    "noisy_label",
    "gold_label"
]
label_map_dict = {
    "negative": 0,
    "positive": 1,
}
df = pd.read_pickle(base_path / "imdb.pkl")

df[label_columns] = df[label_columns].applymap(lambda x: label_map_dict[x])

texts = df.text.values
gold_labels = df.gold_label.values

In [ ]:
df = df.rename(columns={"noisy_label": "realistic"})

In [ ]:
for noise_type in ["uniform", "asymmetric"]:
    for noise_rate in range(5, 55, 5):
        _, df[f"{noise_type}{noise_rate}"] = add_noise(
            texts,
            gold_labels,
            n_classes=2,
            noise_type=noise_type,
            noise_rate=noise_rate / 100
        )

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_outp

In [ ]:
config = {
    "batch_size": 32,
    "show_progress_bar": True,
    "convert_to_numpy": True,
}

model_name = 'sentence-transformers/all-mpnet-base-v2'

embedder = SentenceTransformer(model_name)
embeddings = embedder.encode(texts, **config)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/sentence-transformers/all-mpnet-base-v2 HTTP/1.1" 200 13745
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/5681fe04da6e48e851d5dd1af67367

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 137986289921056 on /root/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/.gitattributes.lock
DEBUG:filelock:Lock 137986289921056 released on /root/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/.gitattributes.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/5681fe04da6e48e851d5dd1af673670cdb299753/1_Pooling/config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 137986289921056 on /root/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/1_Pooling/config.json.lock
DEBUG:filelock:Lock 137986289921056 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/1_Pooling/config.json.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /sentence-transformers/all-mpnet-base-v2/resolve/5681fe04da6e48e851d5dd1af673670cdb299753/1_Pooling/config.json H

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 137986289921056 on /root/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/1_Pooling/config.json.lock
DEBUG:filelock:Lock 137986289921056 released on /root/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/1_Pooling/config.json.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/5681fe04da6e48e851d5dd1af673670cdb299753/README.md HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 137986289925424 on /root/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/README.md.lock
DEBUG:filelock:Lock 137986289925424 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/README.md.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /sentence-transformers/all-mpnet-base-v2/resolve/5681fe04da6e48e851d5dd1af673670cdb299753/README.md HTTP/1.1" 200 10571


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 137986291985392 on /root/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/vocab.txt.lock
DEBUG:filelock:Lock 137986291985392 released on /root/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/vocab.txt.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/5681fe04da6e48e851d5dd1af673670cdb299753/modules.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 137988887232816 on /root/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/modules.json.lock
DEBUG:filelock:Lock 137988887232816 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/modules.json.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /sentence-transformers/all-mpnet-base-v2/resolve/5681fe04da6e48e851d5dd1af673670cdb299753/modules.json HTTP/1.1" 200 349


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 137988887232816 on /root/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/modules.json.lock
DEBUG:filelock:Lock 137988887232816 released on /root/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/modules.json.lock
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


Batches:   0%|          | 0/48 [00:00<?, ?it/s]